In [1]:
import sys
import sklearn
import tensorflow as tf

import cv2
import pandas as pd
import numpy as np

import plotly.graph_objs as go
from plotly.offline import iplot
from matplotlib import pyplot as plt

tf.test.is_gpu_available()

# Set up visualization settings
plt.rc('font', size=14)
plt.rc('axes', labelsize=14, titlesize=14)
plt.rc('legend', fontsize=14)
plt.rc('xtick', labelsize=10)
plt.rc('ytick', labelsize=10)

## Data Visualization

import os

# Function to get data
def get_data():
    return pd.read_csv('metadata.csv/metadata.csv')

meta = get_data()
meta.head()


Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


,videoname,original_width,original_height,label,original
0,aznyksihgl.mp4,129,129,FAKE,xnojggkrxt.mp4
1,gkwmalrvcj.mp4,129,129,FAKE,hqqmtxvbjj.mp4
2,lxnqzocgaq.mp4,223,217,FAKE,xjzkfqddyk.mp4
3,itsbtrrelv.mp4,186,186,FAKE,kqvepwqxfe.mp4
4,ddvgrczjno.mp4,155,155,FAKE,pluadmqqta.mp4


In [2]:
# Reducing dataset size to 16000 images
sample_size = 16000

real_df = meta[meta["label"] == "REAL"].sample(sample_size // 2, random_state=42)
fake_df = meta[meta["label"] == "FAKE"].sample(sample_size // 2, random_state=42)

sample_meta = pd.concat([real_df, fake_df])

from sklearn.model_selection import train_test_split

Train_set, Test_set = train_test_split(sample_meta, test_size=0.2, random_state=42, stratify=sample_meta['label'])
Train_set, Val_set = train_test_split(Train_set, test_size=0.3, random_state=42, stratify=Train_set['label'])

Train_set.shape, Val_set.shape, Test_set.shape


((8960, 5), (3840, 5), (3200, 5))

In [3]:
def retrieve_dataset(set_name):
    images, labels = [], []
    for (img, imclass) in zip(set_name['videoname'], set_name['label']):
        images.append(cv2.imread('faces_224/' + img[:-4] + '.jpg'))
        labels.append(1 if imclass == 'FAKE' else 0)
    return np.array(images), np.array(labels)

X_train, y_train = retrieve_dataset(Train_set)
X_val, y_val = retrieve_dataset(Val_set)
X_test, y_test = retrieve_dataset(Test_set)


In [8]:
from tensorflow.keras.applications import InceptionResNetV2
from tensorflow.keras.layers import Layer, Conv2D, Dense, Flatten
from tensorflow.keras import Model
import tensorflow as tf

# Define Capsule Layer
class CapsuleLayer(Layer):
    def __init__(self, num_capsules=10, dim_capsule=16, **kwargs):
        super(CapsuleLayer, self).__init__(**kwargs)
        self.num_capsules = num_capsules
        self.dim_capsule = dim_capsule

    def build(self, input_shape):
        self.W = self.add_weight(
            shape=[input_shape[-1], self.num_capsules * self.dim_capsule],
            initializer='random_normal',
            trainable=True,
        )
        super(CapsuleLayer, self).build(input_shape)

    def call(self, inputs):
        inputs_expand = tf.expand_dims(inputs, axis=-2)
        inputs_tiled = tf.tile(inputs_expand, [1, 1, self.num_capsules, 1])
        caps = tf.matmul(inputs_tiled, self.W)
        caps = tf.reshape(caps, (-1, caps.shape[1], self.num_capsules, self.dim_capsule))
        return caps

# Build Capsule Network using InceptionResNetV2 base
base_model = InceptionResNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze InceptionResNetV2 layers
for layer in base_model.layers:
    layer.trainable = False

# Capsule Network architecture
x = base_model.output
x = Conv2D(filters=256, kernel_size=3, strides=2, activation='relu')(x)
x = CapsuleLayer()(x)
x = Flatten()(x)
x = Dense(128, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)

# Create the Capsule Network model
model = Model(inputs=base_model.input, outputs=predictions)


ValueError: Exception encountered when calling layer "capsule_layer_4" (type CapsuleLayer).

in user code:

    File "C:\Users\mahes\AppData\Local\Temp\ipykernel_15396\1877425202.py", line 23, in call  *
        inputs_tiled = tf.tile(inputs_expand, [1, 1, self.num_capsules, 1])

    ValueError: Shape must be rank 5 but is rank 4 for '{{node capsule_layer_4/Tile}} = Tile[T=DT_FLOAT, Tmultiples=DT_INT32](capsule_layer_4/ExpandDims, capsule_layer_4/Tile/multiples)' with input shapes: [?,2,2,1,256], [4].


Call arguments received by layer "capsule_layer_4" (type CapsuleLayer):
  • inputs=tf.Tensor(shape=(None, 2, 2, 256), dtype=float32)

In [ ]:
# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, epochs=5, batch_size=16, validation_data=(X_val, y_val))


In [ ]:
# Evaluate the model on the test set
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Loss: {loss:.4f}')
print(f'Test Accuracy: {accuracy:.4f}')


In [ ]:
# Plot model performance
plt.figure(figsize=(15, 5))

# Accuracy plot
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.title('Training and Validation Accuracy')

# Loss plot
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.title('Training and Validation Loss')

plt.tight_layout()
plt.show()
